In [ ]:
import spacy 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words as engwords
nlp = spacy.load("en_core_web_sm")

In [ ]:
def remove_stopwords(words):
    stop_words = stopwords.words('english')
    stop_words.extend(['research', 'subject', 'interest', 'also'])
    new_words = []
    for word in words:
        if word not in stop_words and len(word)>2:
            new_words.append(word)
    return new_words

def remove_non_ascii(words):
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_numbers(words):
    new_words = []
    for word in words:
        if not word.isdigit():
            new_words.append(word)
    return new_words


def normalize(words):
    words = remove_non_ascii(words)
    words = remove_punctuation(words)
    words = remove_numbers(words)
    words = remove_stopwords(words)
    return words

In [ ]:
def getTokens(Text):
    coumpound_nlp = nlp(Text)

    coumpound_word = []
    for X in coumpound_nlp.ents:
        s = re.sub(r'[^\w\s]','',X.text.lower())
        s = re.sub("[\s.;:,*]+", " ", s)
        coumpound_word.append(s)
    
    tokens = word_tokenize(Text.lower())
    fTokens = []
    word = ''
    for t in tokens:
        if word == '':
            word = t
        else:
            word += ' '+t
        find = False
        for X in coumpound_word:
            if word in X:
                find = True
                if word == X:
                    fTokens.append(word)
                    word = ''
                    break
        if find == False:
            tokensW = word_tokenize(word)
            for w in tokensW:
                fTokens.append(w)
            word = ''
    fTokens = normalize(fTokens)
    return fTokens

In [ ]:
import math
from textblob import TextBlob as tb

def tf(word, words):
    return words.count(word) / len(words)

def n_containing(word, wordslist):
    return sum(1 for words in wordslist if word in words)

def idf(word, wordslist):
    return math.log(len(wordslist) / (1 + n_containing(word, wordslist)))

def tfidf(word, words, wordslist):
    return tf(word, words) * idf(word, wordslist)

In [ ]:
def traitement(l0, line1, line2, line3, line4):
    with open('traitement.csv', 'a', newline='', encoding="utf-8") as f:
        words = getTokens(line1)
        #print("Top interest :")
        scores = {word: tf(word, words) for word in words}
        sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        l1 = ' '
        for word, score in sorted_words[:3]:
            l1 += word+" "

        line2 = nlp(line2)

        #print("University :")
        University = []
        for X in line2.ents:
            if 'university' in X.text.lower(): 
                University.append(X.text.lower().replace('\n',''))
        l2 = ' '
        University = set(University)
        for u in University: 
            l2 +=u+" "

        l3  = ' '    
        #print("Advisors :")
        for X in line2.ents:
            if X.label_ == 'PERSON':
                l3  += X.text.lower().replace('\n','')+" "

        l4  = ' '
        #print("laboratories or research teams")
        line3 = nlp(line3)
        for X in line3.ents:
            if X.label_ == 'ORG':
                l4  += X.text.lower().replace('\n','')+" "

        l5 = ' '
        #print("collaborations")
        line4 = nlp(line4)
        collaborations = []
        for X in line4.ents:
            if X.label_ == 'PERSON':
                find = False
                for c in collaborations:
                    if X.text in c:
                        find = True
                if find == False:
                    collaborations.append(X.text.lower().replace("[\s]+",' '))

        collaborations = normalize(set(collaborations))
        for c in collaborations:
            l5  += c+" "

        
        theWriter = csv.writer(f)
        theWriter.writerow([l0, l1, l2, l3, l4, l5])

In [ ]:
#import csv
#with open('CVs.csv', 'w', newline='') as f:
#    theWriter = csv.writer(f)
#    theWriter.writerow(['Name', 'Research Interests', 'Education', 'Research Experience', 'Selected Publications'])

In [ ]:
#import csv
#with open('traitement.csv', 'w', newline='', encoding="utf-8") as f:
#    theWriter = csv.DictWriter(f, fieldnames=["name","interest", "University", "Advisor", "Laboratorie", "Collaboration"])
#    theWriter.writeheader()

In [ ]:
Name = """ """

Research_Interests = """ 

"""

Education = """

"""

Research_Experience = """ 

"""

Selected_Publications = """

"""

In [ ]:
with open('CVs.csv', 'a', newline='', encoding="utf-8") as f:
    theWriter = csv.writer(f)
    theWriter.writerow([Name, Research_Interests, Education, Research_Experience, Selected_Publications])
    traitement(Name, Research_Interests, Education, Research_Experience, Selected_Publications)